In [8]:
from urllib.parse import urlparse
import json

parsed_links = []
count = 0
for u in all_urls:
    parsed = urlparse(u)
    path = parsed.path
    split = parsed.path.split('/')
    product = 0
    data = {
        "url": path,
        "label": 0
    }
    if 'products' in split or 'p' in split:
        data["label"] = 1
        count += 1

    parsed_links.append(data)

In [16]:
import json
with open("dataset.json", "r") as f:
    dataset_urls = json.load(f)
print(f"Loaded {len(dataset_urls)} links.")

Loaded 20113 links.


In [ ]:

# Extract URLs and labels
urls = [item["url"] for item in parsed_links]  # Add domain if needed
labels = [item["label"] for item in parsed_links]

# Preprocess URLs (replace hyphens, slashes, etc.)
def preprocess(url):
    return url.lower().replace("-", " ").replace("/", " ").replace("_", " ")

processed_urls = [preprocess(url) for url in urls]
processed_urls[:5]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF vectorization
vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(3, 5))
X = vectorizer.fit_transform(processed_urls)


In [ ]:
from sklearn.model_selection import train_test_split
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)


In [ ]:
from xgboost import XGBClassifier
# XGBoost model
xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
xgb.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb.predict(X_test)
print("🔍 XGBoost Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
def preprocess_url(url: str) -> str:
    url = url.lower().replace("https://", "").replace("http://", "")
    url = url.replace(".", "/").replace("-", " ").replace("_", " ")
    return url

def predict_url_label(url, model: RandomForestClassifier, vectorizer: TfidfVectorizer) -> int:
    processed = preprocess_url(url)
    vectorized = vectorizer.transform([processed])  # note: list input
    prediction = model.predict(vectorized)[0]
    return prediction

for url in all_urls:
    response = predict_url_label(
        url=url,
        model=rf,
        vectorizer=vectorizer,
    )
    if response == np.int64(0):
        print("Not a product page:", url)
